In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import zipfile
import urllib.request
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = 'cats_and_dogs_filtered'

if not os.path.exists(data_dir):
    print("Downloading Cats and Dogs dataset...")
    dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
    dataset_path = "cats_and_dogs_filtered.zip"
    
    urllib.request.urlretrieve(dataset_url, dataset_path)

    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall()

    print(f"Dataset extracted to {data_dir}")

/home/student/Desktop/220962029/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Dataset extracted to cats_and_dogs_filtered


In [2]:
#question 1

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)  # Adjusted for 256x256 image size
        self.fc2 = nn.Linear(512, 2)
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)  
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),         
])

train_dir = os.path.join(data_dir, 'train')
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01) 

epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")

print("With L2 norm:")

for epoch in range(epochs):
    running_loss = 0.0
    l2_lambda = 0.01  
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param, 2)  
        
        loss += l2_lambda * l2_reg  
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")


Epoch [1/10], Loss: 0.7503085675693694
Epoch [2/10], Loss: 0.693732309909094
Epoch [3/10], Loss: 0.6932253960579161
Epoch [4/10], Loss: 0.6932200600230505
Epoch [5/10], Loss: 0.693252356279464
Epoch [6/10], Loss: 0.6932271671673608
Epoch [7/10], Loss: 0.6932206664766584
Epoch [8/10], Loss: 0.6932709737429543
Epoch [9/10], Loss: 0.6932072885452755
Epoch [10/10], Loss: 0.6931886985188439
With L2 norm:
Epoch [1/10], Loss: 0.7519503699408637
Epoch [2/10], Loss: 0.7403952346907722
Epoch [3/10], Loss: 0.7336394654379951
Epoch [4/10], Loss: 0.7258287583078656
Epoch [5/10], Loss: 0.72342782739609
Epoch [6/10], Loss: 0.7186134883335659
Epoch [7/10], Loss: 0.710716169977945
Epoch [8/10], Loss: 0.7068073531937977
Epoch [9/10], Loss: 0.7039466774652875
Epoch [10/10], Loss: 0.7020383997569009


In [3]:
#question 2

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)  
        self.fc2 = nn.Linear(512, 2)
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)  
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),          
])

train_dir = os.path.join(data_dir, 'train')
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  

print("Training with L1 regularization (Optimizer)")
l1_lambda = 0.01  
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  

        l1_reg = 0
        for param in model.parameters():
            l1_reg += torch.sum(torch.abs(param))  
        
        loss += l1_lambda * l1_reg  
        
        loss.backward() 
        optimizer.step()  

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")



Training with L1 regularization (Optimizer)
Epoch [1/10], Loss: 165.08566538492838
Epoch [2/10], Loss: 90.13900466192337
Epoch [3/10], Loss: 89.55810583205451
Epoch [4/10], Loss: 89.67102232433501
Epoch [5/10], Loss: 89.50572228810144
Epoch [6/10], Loss: 89.59697105771019
Epoch [7/10], Loss: 89.74316442580451
Epoch [8/10], Loss: 89.45288546123201
Epoch [9/10], Loss: 89.50607324024988
Epoch [10/10], Loss: 89.60752347915891


In [ ]:
#question 3

class SimpleCNNWithDropout(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(SimpleCNNWithDropout, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)  
        self.fc2 = nn.Linear(512, 2)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.dropout = nn.Dropout(dropout_rate) if dropout_rate > 0 else nn.Identity()  
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32) 
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),          
])

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

def train_model(dropout_rate=0.0, epochs=10):
    model = SimpleCNNWithDropout(dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  
            outputs = model(inputs) 
            loss = criterion(outputs, labels) 

            loss.backward() 
            optimizer.step()  

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)  

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Validation Loss: {val_loss:.4f}")

print("Training with Dropout Regularization (Dropout rate = 0.5):")
train_model(dropout_rate=0.5)

print("\nTraining without Dropout Regularization:")
train_model(dropout_rate=0.0)


Training with Dropout Regularization (Dropout rate = 0.5):
Epoch [1/10], Train Loss: 0.8049, Validation Loss: 0.6932
Epoch [2/10], Train Loss: 0.6938, Validation Loss: 0.6934
Epoch [3/10], Train Loss: 0.6931, Validation Loss: 0.6926
Epoch [4/10], Train Loss: 0.6921, Validation Loss: 0.6917
Epoch [5/10], Train Loss: 0.6933, Validation Loss: 0.6922
Epoch [6/10], Train Loss: 0.6919, Validation Loss: 0.6918
Epoch [7/10], Train Loss: 0.6895, Validation Loss: 0.6747
Epoch [8/10], Train Loss: 0.6862, Validation Loss: 0.6876


In [ ]:
# question 4

class CustomDropout(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(CustomDropout, self).__init__()
        self.dropout_rate = dropout_rate
    
    def forward(self, x):
        if self.training:  
            # Bernoulli distribution: 1 - dropout_rate gives us the probability of "keeping" a neuron
            mask = torch.bernoulli(torch.full(x.shape, 1 - self.dropout_rate, device=x.device))
            x = x * mask 
            x = x / (1 - self.dropout_rate)  
        return x

class SimpleCNNWithCustomDropout(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(SimpleCNNWithCustomDropout, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)  
        self.fc2 = nn.Linear(512, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.custom_dropout = CustomDropout(dropout_rate) 
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)  
        x = torch.relu(self.fc1(x))
        x = self.custom_dropout(x)  
        x = self.fc2(x)
        return x

class SimpleCNNWithLibraryDropout(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(SimpleCNNWithLibraryDropout, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)  
        self.fc2 = nn.Linear(512, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_rate)  
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)  
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),         
])

train_dir = os.path.join(data_dir, 'train')
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

def train_model(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    epochs = 10
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  
            outputs = model(inputs)  
            loss = criterion(outputs, labels)  

            loss.backward()  
            optimizer.step()  

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")

print("Training with Custom Dropout Regularization:")
model_with_custom_dropout = SimpleCNNWithCustomDropout(dropout_rate=0.5).to(device)
train_model(model_with_custom_dropout)



In [ ]:
#questions 5

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*64*64, 512)
        self.fc2 = nn.Linear(512, 2)
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64*64*64)  
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),          
])

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.01):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def train_with_early_stopping(model, criterion, optimizer, epochs=20):
    early_stopping = EarlyStopping(patience=5, min_delta=0.01)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  
            outputs = model(inputs)  
            loss = criterion(outputs, labels) 

            loss.backward()  
            optimizer.step()  

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)  

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Validation Loss: {val_loss:.4f}")

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

model_with_early_stopping = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_with_early_stopping.parameters(), lr=0.001)

print("Training with Early Stopping:")
train_with_early_stopping(model_with_early_stopping, criterion, optimizer, epochs=10)
